# Objective-Channel Incompatibility — Minimal Simulation

This notebook reproduces a minimal diagnostic simulation showing that
optimization under restricted objective channels may lead to structurally
incompatible parameter estimates compared to full trial-level likelihood optimization.

Associated Zenodo technical note:
DOI: 10.5281/zenodo.18632836

In [ ]:
import numpy as np
import pandas as pd
from pyddm import Model, Fittable, Sample
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision
from pyddm.functions import fit_adjust_model
import warnings
warnings.filterwarnings("ignore")

## True generative model

In [ ]:
true_model = Model(
    drift=DriftConstant(drift=0.8),
    noise=NoiseConstant(noise=1),
    bound=BoundConstant(B=1.2),
    overlay=OverlayNonDecision(nondectime=0.25),
    dx=0.005, dt=0.01, T_dur=2.5
)

sample = true_model.simulate_n_trials(800)
sample = Sample.from_pandas_dataframe(sample)

## Full likelihood fit (M0)

In [ ]:
model_full = Model(
    drift=DriftConstant(drift=Fittable(minval=-3, maxval=3)),
    noise=NoiseConstant(noise=1),
    bound=BoundConstant(B=Fittable(minval=0.5, maxval=3)),
    overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=0.5)),
    dx=0.005, dt=0.01, T_dur=2.5
)

fit_adjust_model(model=model_full, sample=sample)
print("Full fit parameters:")
print(model_full.parameters())

## Accuracy-only objective fit (M1)

In [ ]:
def accuracy_only_loss(model, sample):
    sol = model.solve()
    pred_acc = sol.prob("correct")
    obs_acc = sample.prob_correct()
    return (pred_acc - obs_acc)**2

model_acc = Model(
    drift=DriftConstant(drift=Fittable(minval=-3, maxval=3)),
    noise=NoiseConstant(noise=1),
    bound=BoundConstant(B=Fittable(minval=0.5, maxval=3)),
    overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=0.5)),
    dx=0.005, dt=0.01, T_dur=2.5
)

fit_adjust_model(model=model_acc, sample=sample, lossfunction=accuracy_only_loss)
print("Accuracy-only fit parameters:")
print(model_acc.parameters())